In [ ]:
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER,LATITUDE_FORMATTER
import os,errno
import sys
import cartopy.feature as cfeature
import matplotlib.ticker as mticker
import pandas as pd
import datetime as dt
from scipy.ndimage.measurements import label
from math import sin, cos, sqrt, atan2, radians
import geopy.distance
%matplotlib inline

dir2='/thorncroftlab_rit/ahenny/rain/'
dir1='/thorncroftlab_rit/ahenny/rain/US/ghcnd_all/'
dir='/thorncroftlab_rit/ahenny/rain/DISSERTATION_SCRIPTS_RESULTS/'

#Finds dates associated with each weather type and plots composites

In [ ]:
ds=xr.open_dataset(dir+'extreme_days_ghcnd_99station_80area95.nc')
lats=ds['lats'].values.tolist()
lons=ds['lons'].values.tolist()
lons=[x+360. for x in lons]
dates=ds['dates'].values
dates_unique=list(set(dates))
dates_unique=pd.DatetimeIndex(dates_unique).sort_values()
stations=ds['stations'].values.tolist()
obs=ds['obs'].values.tolist()
print(dates)

ds1=xr.open_dataset(dir+'ivt_basics_neusa.nc')
ivt_threshold=ds1['ivt_threshold']
ivt_mag=ds1['ivt_mag']
ivtx=ds1['ivtx']
ivty=ds1['ivty']
print(ivtx.time)

lon_range=np.arange(-125,-30,0.25)
lat_range=np.arange(15,60,0.25)
if 1==1:
    ds3=xr.open_dataset(dir+'era_5_fall_u_850_neusa.nc')
    u_850=ds3['u'].sel(latitude=lat_range,longitude=lon_range)
    ds8=xr.open_dataset(dir+'era_5_fall_v_850_neusa.nc')
    v_850=ds8['v'].sel(latitude=lat_range,longitude=lon_range)
    ds5=xr.open_dataset(dir+'era_5_fall_mslp_neusa.nc')
    print(ds5)
    slp=ds5['msl'].sel(latitude=lat_range,longitude=lon_range)

    ds6=xr.open_dataset(dir+'era_5_fall_sfc_remnants_neusa.nc')
    print(ds6)
    ds6['longitude']=ds6.longitude.values-180
    slp_2=ds6['mslp'].sel(latitude=lat_range,longitude=lon_range)
    ds7=xr.open_dataset(dir+'era_5_fall_pl_remnants_neusa.nc')
    ds7['longitude']=ds7.longitude.values-180
    u_850_2=ds7['u'].sel(latitude=lat_range,longitude=lon_range)
    v_850_2=ds7['v'].sel(latitude=lat_range,longitude=lon_range)

ds4=xr.open_dataset(dir+'neusa_ep_days_stats_var95const_station95.nc')
ar_yesno4=ds4['ar_yesno'].values.tolist()
ivt_yesno4=ds4['ivt_yesno'].values.tolist()
tc_yesno4=ds4['tc_yesno'].values.tolist()
other_yesno4=ds4['other_yesno'].values.tolist()
tc_linked_ar_yesno4=ds4['tc_linked_ar_yesno'].values.tolist()
tc_linked_ivt_yesno4=ds4['tc_linked_ivt_yesno'].values.tolist()
tc_remnant_linked_ivt_yesno4=ds4['ivt_tc_remnants_yesno'].values.tolist()
tc_remnant_ar_combo_yesno4=ds4['ar_tc_remnant_combo_yesno'].values.tolist()
tc_remnant_linked_ar_yesno4=ds4['tc_remnant_linked_ar_yesno'].values.tolist()
tc_remnants_yesno4=ds4['tc_remnants_yesno'].values.tolist()
tc_ar_combo_yesno4=ds4['tc_ar_combo_yesno'].values.tolist()
print(sum(ar_yesno4))

In [ ]:
zipped_t1=list(zip(dates_unique,ar_yesno4))#ar-related = green
zipped_t2=list(zip(dates_unique,tc_linked_ar_yesno4))
zipped_t3=list(zip(dates_unique,tc_remnant_linked_ar_yesno4))
zipped_t4=list(zip(dates_unique,tc_yesno4))#tc-related = blue
zipped_t5=list(zip(dates_unique,tc_ar_combo_yesno4))
zipped_t6=list(zip(dates_unique,tc_remnant_ar_combo_yesno4))
zipped_t7=list(zip(dates_unique,tc_remnants_yesno4))
zipped_t8=list(zip(dates_unique,ivt_yesno4))#other IVT-related = grey
zipped_t9=list(zip(dates_unique,tc_linked_ivt_yesno4))
zipped_t10=list(zip(dates_unique,tc_remnant_linked_ivt_yesno4))
zipped_t11=list(zip(dates_unique,other_yesno4))#unspecified = brown

dates_t1=[x[0] for x in zipped_t1 if x[1]==1]
dates_t2=[x[0] for x in zipped_t2 if x[1]==1]
dates_t3=[x[0] for x in zipped_t3 if x[1]==1]
dates_t4=[x[0] for x in zipped_t4 if x[1]==1]
dates_t5=[x[0] for x in zipped_t5 if x[1]==1]
dates_t6=[x[0] for x in zipped_t6 if x[1]==1]
dates_t7=[x[0] for x in zipped_t7 if x[1]==1]
dates_t8=[x[0] for x in zipped_t8 if x[1]==1]
dates_t9=[x[0] for x in zipped_t9 if x[1]==1]
dates_t10=[x[0] for x in zipped_t10 if x[1]==1]
dates_t11=[x[0] for x in zipped_t11 if x[1]==1]

dates_t1=[pd.to_datetime(x)+dt.timedelta(hours=12) for x in dates_t1]
dates_t2=[pd.to_datetime(x)+dt.timedelta(hours=12) for x in dates_t2]
dates_t3=[pd.to_datetime(x)+dt.timedelta(hours=12) for x in dates_t3]
dates_t4=[pd.to_datetime(x)+dt.timedelta(hours=12) for x in dates_t4]
dates_t5=[pd.to_datetime(x)+dt.timedelta(hours=12) for x in dates_t5]
dates_t6=[pd.to_datetime(x)+dt.timedelta(hours=12) for x in dates_t6]
dates_t7=[pd.to_datetime(x)+dt.timedelta(hours=12) for x in dates_t7]
dates_t8=[pd.to_datetime(x)+dt.timedelta(hours=12) for x in dates_t8]
dates_t9=[pd.to_datetime(x)+dt.timedelta(hours=12) for x in dates_t9]
dates_t10=[pd.to_datetime(x)+dt.timedelta(hours=12) for x in dates_t10]
dates_t11=[pd.to_datetime(x)+dt.timedelta(hours=12) for x in dates_t11]

dates_ar=dates_t1+dates_t2+dates_t3
dates_tc=dates_t4+dates_t5+dates_t6+dates_t7
dates_other=dates_t8+dates_t9+dates_t10+dates_t11

dates_tc=pd.DatetimeIndex(dates_tc)
dates_tc_first=[x for x in dates_tc if x.year<=1999]
dates_tc_second=[x for x in dates_tc if x.year>=2000]
print(dates_tc_first)

dates_unique_first=[x for x in dates_unique if x.year<=1999]
dates_unique_second=[x for x in dates_unique if x.year>=2000]

dates_ar=pd.DatetimeIndex(dates_ar)
dates_ar_first=[x for x in dates_ar if x.year<=1999]
dates_ar_second=[x for x in dates_ar if x.year>=2000]

print(dates_t1)
print(dates_t2)
print(dates_t3)
print(dates_t4)
print(dates_t5)
print(dates_t6)
print(dates_t7)
print(dates_t8)
print(dates_t9)
print(dates_t10)
print(dates_t11)

print(dates_tc)
print(dates_ar)
print(dates_other)

In [ ]:
ivt_mag_sel=ivt_mag.sel(time=dates_t4)
u_850_sel=u_850.sel(time=dates_t4)
v_850_sel=v_850.sel(time=dates_t4)
slp_sel=slp.sel(time=dates_t4)
ivt_mean=ivt_mag_sel.mean(dim='time',skipna=True)
u_mean=u_850_sel.mean(dim='time',skipna=True)
v_mean=v_850_sel.mean(dim='time',skipna=True)
slp_mean=slp_sel.mean(dim='time',skipna=True)

In [ ]:
#to draw grid lines on Lambert Conformal projection; 
#CREDIT ajdawson on GitHub https://gist.github.com/ajdawson/dd536f786741e987ae4e

from copy import copy
import shapely.geometry as sgeom
def find_side(ls, side):
    """
    Given a shapely LineString which is assumed to be rectangular, return the
    line corresponding to a given side of the rectangle.
    
    """
    minx, miny, maxx, maxy = ls.bounds
    points = {'left': [(minx, miny), (minx, maxy)],
              'right': [(maxx, miny), (maxx, maxy)],
              'bottom': [(minx, miny), (maxx, miny)],
              'top': [(minx, maxy), (maxx, maxy)],}
    return sgeom.LineString(points[side])


def lambert_xticks(ax, ticks):
    """Draw ticks on the bottom x-axis of a Lambert Conformal projection."""
    te = lambda xy: xy[0]
    lc = lambda t, n, b: np.vstack((np.zeros(n) + t, np.linspace(b[2], b[3], n))).T
    xticks, xticklabels = _lambert_ticks(ax, ticks, 'bottom', lc, te)
    ax.xaxis.tick_bottom()
    ax.set_xticks(xticks)
    ax.set_xticklabels([ax.xaxis.get_major_formatter()(xtick) for xtick in xticklabels])
    

def lambert_yticks(ax, ticks):
    """Draw ricks on the left y-axis of a Lamber Conformal projection."""
    te = lambda xy: xy[1]
    lc = lambda t, n, b: np.vstack((np.linspace(b[0], b[1], n), np.zeros(n) + t)).T
    yticks, yticklabels = _lambert_ticks(ax, ticks, 'left', lc, te)
    ax.yaxis.tick_left()
    ax.set_yticks(yticks)
    ax.set_yticklabels([ax.yaxis.get_major_formatter()(ytick) for ytick in yticklabels])
def _lambert_ticks(ax, ticks, tick_location, line_constructor, tick_extractor):
    """Get the tick locations and labels for an axis of a Lambert Conformal projection."""
    outline_patch = sgeom.LineString(ax.outline_patch.get_path().vertices.tolist())
    axis = find_side(outline_patch, tick_location)
    n_steps = 30
    extent = ax.get_extent(ccrs.PlateCarree())
    _ticks = []
    for t in ticks:
        xy = line_constructor(t, n_steps, extent)
        proj_xyz = ax.projection.transform_points(ccrs.Geodetic(), xy[:, 0], xy[:, 1])
        xyt = proj_xyz[..., :2]
        ls = sgeom.LineString(xyt.tolist())
        locs = axis.intersection(ls)
        if not locs:
            tick = [None]
        else:
            tick = tick_extractor(locs.xy)
        _ticks.append(tick[0])
    # Remove ticks that aren't visible:    
    ticklabels = copy(ticks)
    while True:
        try:
            index = _ticks.index(None)
        except ValueError:
            break
        _ticks.pop(index)
        ticklabels.pop(index)
    return _ticks, ticklabels

In [ ]:
dsa=xr.open_dataset(dir+'neusa_ar_climo_fall.nc')
ar=dsa.ar
tc_trop=dsa.tc_trop
tc_nontrop=dsa.tc_nontrop
labeled=dsa.labeled

In [ ]:
ar_sel=ar.sel(time=dates_t4).fillna(0)
ar_mean=ar_sel.mean(dim='time',skipna=True)
ar_mean=ar_mean.where(ar_mean>=0.4)
ar_mean=ar_mean/ar_mean
ar_mean['longitude']=ar_mean.longitude.values-360

In [ ]:
import matplotlib as mpl
month_dict={'1':'January','2':'February','3':'March','4':'April','5':'May','6':'June','7':'July','8':'August','9':'September','10':'October','11':'November','12':'December'}    
clon=-70
clat=35
proj_map = ccrs.LambertConformal(central_longitude=clon, central_latitude=clat)
fig = plt.figure(figsize=(24,16))
ax=plt.subplot(1,1,1,projection=proj_map)
ax.coastlines(resolution='10m')
ax.add_feature(cfeature.STATES.with_scale('10m'),alpha=0.3)
ax.add_feature(cfeature.LAKES.with_scale('50m'))
countries = cfeature.NaturalEarthFeature(category='cultural',name='admin_0_boundary_lines_land',scale='50m',facecolor='none')
ax.add_feature(countries)
ax.set_extent([-100,-45,15,55],crs=ccrs.PlateCarree())

# *must* call draw in order to get the axis boundary used to add ticks:
fig.canvas.draw()

# Define gridline locations and draw the lines using cartopy's built-in gridliner:
xticks = [-135,-130,-125,-120,-115,-110,-105,-95,-85,-75,-65,-55,-45,-40,-35,-30,-25,-20]
yticks = [5,10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80]
ax.gridlines(xlocs=xticks, ylocs=yticks,alpha=0.5)
ax.tick_params(labelsize=24)
# Label the end-points of the gridlines using the custom tick makers:
ax.xaxis.set_major_formatter(LONGITUDE_FORMATTER) 
ax.yaxis.set_major_formatter(LATITUDE_FORMATTER)
lambert_xticks(ax, xticks)
lambert_yticks(ax, yticks)

cax1=ax.contourf(ivt_mean.longitude,ivt_mean.latitude,ivt_mean,levels=np.arange(250,800,40),extend='both',transform=ccrs.PlateCarree(),cmap=plt.cm.Greys,alpha=0.5)
cbar=plt.colorbar(cax1,pad=0,fraction=0.046)
cbar.set_label('kg/m/s',fontsize=26,rotation=90,labelpad=15)
cbar.ax.tick_params(labelsize=24)

c=ax.contour(slp_mean.longitude,slp_mean.latitude,slp_mean/100.,transform=ccrs.PlateCarree(),colors='k',levels=np.arange(960,1040,4))
plt.clabel(c,fmt='%.00f',fontsize=19)
X=8
q=ax.quiver(u_mean.longitude[0::X].values,u_mean.latitude[0::X].values,u_mean[0::X,0::X].values,v_mean[0::X,0::X].values,transform=ccrs.PlateCarree(),units='inches',scale=20,width=0.045,alpha=0.5)
ax.quiverkey(q,0.94,1.01,10,'10 m/s',fontproperties={'size':18})
ax.set_title('Pure TC',fontsize=44,pad=10)

cax2=ax.contourf(ar_mean.longitude,ar_mean.latitude,ar_mean,[0,1],colors='b',transform=ccrs.PlateCarree(),hatches=[None,'.'],alpha=0.1,label='AR probability >= 0.5',zorder=20)    
mpl.rcParams['hatch.linewidth']=0.01
props = dict(boxstyle='round', facecolor='wheat', alpha=1.0)
ax.text(0.01, 0.9875,'n='+str(len(dates_t4)), transform=ax.transAxes, fontsize=28,verticalalignment='top', bbox=props,zorder=30)
plt.show()

In [ ]:
fig.savefig(dir+'neusa_wtype_95composite_111.png')